# Import libraries

In [4]:
import sys
sys.path.append("../src/")
import os

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [5]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler

In [6]:
from tqdm.auto import tqdm
# Preprocessing
from scipy.signal import butter, lfilter
from tsflex.processing import SeriesPipeline, SeriesProcessor
# Extract Features
import antropy as ant
import scipy.stats as ss
from yasa import bandpower
import scipy.stats as ss
from tsflex.features import (
    FeatureCollection,
    FuncWrapper,
    MultipleFeatureDescriptors,
    FuncWrapper,
)
from tsflex.features.integrations import tsfresh_settings_wrapper

In [7]:
from data import load_signals, load_annotations, annotation_to_30s_labels

# Extract Features

## Load Data

In [8]:
data_folder = "../data/clips/"
dfs = []
sorted_files = sorted(os.listdir(data_folder))
psg_hypnogram_files = [(p, h) for p, h in zip(sorted_files[::2], sorted_files[1:][::2])]
df_files = pd.DataFrame(psg_hypnogram_files, columns=["psg_file", "label_file"])
df_files["patient_id"] = df_files.psg_file.apply(lambda f: f[:5])
df_files["file_id"] = df_files.psg_file.apply(lambda f: f[:6])
df_files["clip_id"] = df_files.psg_file.apply(lambda f: f[:7])
df_files

,psg_file,label_file,patient_id,file_id,clip_id
0,SC400100E0-PSG.edf,SC400100EC-Hypnogram.edf,SC400,SC4001,SC40010
1,SC400101E0-PSG.edf,SC400101EC-Hypnogram.edf,SC400,SC4001,SC40010
2,SC400102E0-PSG.edf,SC400102EC-Hypnogram.edf,SC400,SC4001,SC40010
3,SC400103E0-PSG.edf,SC400103EC-Hypnogram.edf,SC400,SC4001,SC40010
4,SC400104E0-PSG.edf,SC400104EC-Hypnogram.edf,SC400,SC4001,SC40010
...,...,...,...,...,...
3299,SC482226G0-PSG.edf,SC482226GC-Hypnogram.edf,SC482,SC4822,SC48222
3300,SC482227G0-PSG.edf,SC482227GC-Hypnogram.edf,SC482,SC4822,SC48222
3301,SC482228G0-PSG.edf,SC482228GC-Hypnogram.edf,SC482,SC4822,SC48222
3302,SC482229G0-PSG.edf,SC482229GC-Hypnogram.edf,SC482,SC4822,SC48222


In [9]:
PATIENT_IDX = 0
signal_dict = {}

for idx in range(len(df_files)):
    sig, freq = load_signals(
        data_folder + "/" + df_files.iloc[idx].psg_file,
        only_info=True,
    )
    for s, f in zip(sig, freq):
        if (s, f) in signal_dict.keys():
            signal_dict[(s, f)].append(idx)
        else:
            signal_dict[(s, f)] = [idx]

common_signals = [
    'EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal',  # All have the same sampling rate (100 Hz)
    'EMG submental',  # Also 100 Hz
]

data = load_signals(
    data_folder + "/" + df_files.iloc[PATIENT_IDX].psg_file,
    retrieve_signals=common_signals,
)
annotations = load_annotations(
    data_folder + "/" + df_files.iloc[PATIENT_IDX].label_file,
    data_folder + "/" + df_files.iloc[PATIENT_IDX].psg_file,
)

### Plot 1 Patient's Data

In [ ]:
# Used for plotting
cats = [
    "Sleep stage ?",
    "Movement time",
    "Sleep stage W",
    "Sleep stage 1",
    "Sleep stage 2",
    "Sleep stage 3",
    "Sleep stage 4",
    "Sleep stage R",
][::-1]

idxs = [s.name for s in data]

fig = FigureResampler(
    make_subplots(
        rows=len(idxs) + 1,
        cols=1,
        shared_xaxes=True,
        subplot_titles=idxs + ["Hypnogram"],
        vertical_spacing=0.05,
    ),
    default_n_shown_samples=3_000,
)

for idx, s in enumerate(data, 1):
    fig.add_trace(
        go.Scattergl(
            x=[], y=[], name=s.name, line_width=1,
        ),
        hf_x=s.index,
        hf_y=s.values,
        row=idx,
        col=1,
    )

fig.add_trace(
    go.Scattergl(
        x=[], y=[], name="Hypnogram", line_width=1.5, line_shape="hv",
    ),
    hf_x=annotation_to_30s_labels(annotations).index,
    hf_y=annotation_to_30s_labels(annotations)["label"].astype("category"),
    row=len(idxs) + 1,
    col=1,
)

fig.update_layout(height=900)
fig.update_yaxes(categoryorder="array", categoryarray=cats)

fig.update_layout(
    legend_traceorder="normal",
    template="plotly_white",
    #     legend=dict(orientation="h", y=1.07, xanchor="right", x=1),
)

fig.show_dash(port=8043)

## Preprocess by bandpass filter

In [10]:
def butter_bandpass_filter(sig, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype="band")
    y = lfilter(b, a, sig)
    return y

eeg_bandpass = SeriesProcessor(
    function=butter_bandpass_filter,
    series_names=["EEG Fpz-Cz", "EEG Pz-Oz", "EOG horizontal"],
    lowcut=0.4,
    highcut=30,
    fs=100,
)

emg_bandpass = SeriesProcessor(
    function=butter_bandpass_filter,
    series_names=["EMG submental"],
    lowcut=0.5,
    highcut=10,
    fs=100,
)

process_pipe = SeriesPipeline([eeg_bandpass, emg_bandpass])
data_processed = process_pipe.process(data, return_all_series=True)

08-Jun-23 12:28:05 | INFO | Finished function [butter_bandpass_filter] on [('EEG Fpz-Cz',), ('EEG Pz-Oz',), ('EOG horizontal',)] in [0.006893634796142578 seconds]!
08-Jun-23 12:28:05 | INFO | Finished function [butter_bandpass_filter] on [('EMG submental',)] in [0.0006701946258544922 seconds]!


### Plot 1 Patient's Preprocessed data
The cell "Plot 1 Patient's Data" should be executed first.

In [ ]:
for s in data_processed:
    idx = idxs.index(s.name.split("_")[-1]) + 1
    fig.add_trace(
        go.Scatter(x=[], y=[], name=s.name + " processed", line_width=1),
        hf_x=s.index,
        hf_y=s.values,
        row=idx,
        col=1,
    )
    
fig.show_dash(port=8043)

# Extract Features from all Patients

## Feature Extraction

In [11]:
tsfresh_settings = {
    "fft_aggregated": [
        {"aggtype": "centroid"},
        {"aggtype": "variance"},
        {"aggtype": "skew"},
        {"aggtype": "kurtosis"},
    ],
    "fourier_entropy": [
        {"bins": 2},
        {"bins": 3},
        {"bins": 5},
        {"bins": 10},
        {"bins": 30},
        {"bins": 60},
        {"bins": 100},
    ],
    "binned_entropy": [
        {"max_bins": 5},
        {"max_bins": 10},
        {"max_bins": 30},
        {"max_bins": 60},
    ],
}

def wrapped_higuchi_fd(x):
    x = np.array(x, dtype="float64")
    return ant.higuchi_fd(x)

def wrapped_bandpowers(x, sf, bands):
    return bandpower(x, sf=sf, bands=bands).values[0][:-2]

bands = [
    (0.4, 1, "sdelta"),
    (1, 4, "fdelta"),
    (4, 8, "theta"),
    (8, 12, "alpha"),
    (12, 16, "sigma"),
    (16, 30, "beta"),
]
bandpowers_ouputs = [b[2] for b in bands] + ["TotalAbsPow"]

In [12]:
time_funcs = [
    np.std,
    ss.iqr,
    ss.skew,
    ss.kurtosis,
    ant.num_zerocross,
    FuncWrapper(
        ant.hjorth_params, output_names=["horth_mobility", "hjorth_complexity"]
    ),
    wrapped_higuchi_fd,
    ant.petrosian_fd,
    ant.perm_entropy,
] + tsfresh_settings_wrapper(tsfresh_settings)

freq_funcs = [
    FuncWrapper(wrapped_bandpowers, sf=100, bands=bands, output_names=bandpowers_ouputs)
]

time_feats = MultipleFeatureDescriptors(
    time_funcs,
    ["EEG Fpz-Cz", "EEG Pz-Oz", "EOG horizontal", "EMG submental"],
    windows=["30s", "60s", "90s"],
    strides="30s",
)
freq_feats = MultipleFeatureDescriptors(
    freq_funcs,
    ["EEG Fpz-Cz", "EEG Pz-Oz", "EOG horizontal"],
    windows=["30s", "60s", "90s"],
    strides="30s",
)

feature_collection = FeatureCollection([time_feats, freq_feats])
feats = feature_collection.calculate(data_processed, return_df=True, show_progress=True)

  0%|          | 0/261 [00:00<?, ?it/s]

08-Jun-23 12:28:10 | INFO | Finished function [skew] on [('EEG Fpz-Cz',)] with window-stride [0 days 00:00:30, 0 days 00:00:30] in [0.005666971206665039 seconds]!
08-Jun-23 12:28:10 | INFO | Finished function [wrapped_higuchi_fd] on [('EEG Fpz-Cz',)] with window-stride [0 days 00:00:30, 0 days 00:00:30] in [0.00469517707824707 seconds]!
08-Jun-23 12:28:10 | INFO | Finished function [perm_entropy] on [('EEG Fpz-Cz',)] with window-stride [0 days 00:00:30, 0 days 00:00:30] in [0.012369155883789062 seconds]!
08-Jun-23 12:28:10 | INFO | Finished function [hjorth_params] on [('EEG Fpz-Cz',)] with window-stride [0 days 00:00:30, 0 days 00:00:30] in [0.003161191940307617 seconds]!
08-Jun-23 12:28:10 | INFO | Finished function [std] on [('EEG Fpz-Cz',)] with window-stride [0 days 00:00:30, 0 days 00:00:30] in [0.0013949871063232422 seconds]!
08-Jun-23 12:28:10 | INFO | Finished function [iqr] on [('EEG Fpz-Cz',)] with window-stride [0 days 00:00:30, 0 days 00:00:30] in [0.004609107971191406 sec

## Pipeline for all Patients

In [18]:
df_feats = []
for psg_file, hypnogram_file in tqdm(zip(df_files.psg_file, df_files.label_file)):
    file_folder = data_folder + "/"
    # Load the data, process the data and extract features
    data = load_signals(file_folder + psg_file)
    data_processed = process_pipe.process(data)
    df_feat = feature_collection.calculate(
        data_processed, return_df=True, window_idx="begin"
    ).astype("float32")
    # Add the labels (and reduce features to only data for which we have labels)
    annotations = load_annotations(file_folder + hypnogram_file, file_folder + psg_file)
    annotations = annotation_to_30s_labels(annotations)
    df_feat = df_feat.merge(annotations, left_index=True, right_index=True)
    # Add the file name & folder
    df_feat["psg_file"] = psg_file
    df_feat["patient_id"] = psg_file[:5]
    df_feat["file_id"] = psg_file[:6]
    df_feat["clip_id"] = psg_file[:7]
    # Collect the dataframes
    df_feats += [df_feat]

df_feats = pd.concat(df_feats)
df_feats.rename(columns={"description": "label"}, inplace=True)

0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7fa650367d30>
Traceback (most recent call last):
  File "/Users/wangyanting/opt/anaconda3/envs/BCI_2023/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


In [ ]:
df_feats.to_csv('/caches/cache_clip.csv')
df_feats.to_parquet("../features/sleep-edf_cassette_clip_features_ALL_90s.parquet")